In [12]:
#데이터셋을 만들어 주는 함수
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.layers import LSTM, GRU, Dropout
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

df = pd.read_csv("[IT]삼성전자.csv")
df_columns = ['Date','Open', 'High', 'Low', 'Close', 'Volume']
df.columns = df_columns
df = df.sort_values('Date')
df['Date']= df['Date'].astype('str')
from datetime import datetime
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace=True)

ma = [5,20,60,120]
for days in ma:
    df['ma_'+str(days)] = df['Close'].rolling(window = days).mean()
    df['vma_'+str(days)] = df['Volume'].rolling(window = days).mean()

df.dropna(inplace=True)


import ta

H, L, C, V = df['High'], df['Low'], df['Close'], df['Volume']

# df['bol_high'] = ta.volatility.bollinger_hband(C)
# df['bol_low']  = ta.volatility.bollinger_lband(C)
df['MFI'] = ta.volume.money_flow_index(
    high=H, low=L, close=C, volume=V, fillna=True)

df['ADI'] = ta.volume.acc_dist_index(
    high=H, low=L, close=C, volume=V, fillna=True)

df['OBV'] = ta.volume.on_balance_volume(close=C, volume=V, fillna=True)
df['CMF'] = ta.volume.chaikin_money_flow(
    high=H, low=L, close=C, volume=V, fillna=True)

df['FI'] = ta.volume.force_index(close=C, volume=V, fillna=True)
df['EOM, EMV'] = ta.volume.ease_of_movement(
    high=H, low=L, volume=V, fillna=True)

df['VPT'] = ta.volume.volume_price_trend(close=C, volume=V, fillna=True)
df['NVI'] = ta.volume.negative_volume_index(close=C, volume=V, fillna=True)
df['VMAP'] = ta.volume.volume_weighted_average_price(
    high=H, low=L, close=C, volume=V, fillna=True)

# Volatility
df['ATR'] = ta.volatility.average_true_range(
    high=H, low=L, close=C, fillna=True)
df['BHB'] = ta.volatility.bollinger_hband(close=C, fillna=True)
df['BLB'] = ta.volatility.bollinger_lband(close=C, fillna=True)
df['KCH'] = ta.volatility.keltner_channel_hband(
    high=H, low=L, close=C, fillna=True)
df['KCL'] = ta.volatility.keltner_channel_lband(
    high=H, low=L, close=C, fillna=True)
df['KCM'] = ta.volatility.keltner_channel_mband(
    high=H, low=L, close=C, fillna=True)
df['DCH'] = ta.volatility.donchian_channel_hband(
    high=H, low=L, close=C, fillna=True)
df['DCL'] = ta.volatility.donchian_channel_lband(
    high=H, low=L, close=C, fillna=True)
df['DCM'] = ta.volatility.donchian_channel_mband(
    high=H, low=L, close=C, fillna=True)
df['UI'] = ta.volatility.ulcer_index(close=C, fillna=True)
# Trend
df['SMA'] = ta.trend.sma_indicator(close=C, fillna=True)
df['EMA'] = ta.trend.ema_indicator(close=C, fillna=True)
df['WMA'] = ta.trend.wma_indicator(close=C, fillna=True)
df['MACD'] = ta.trend.macd(close=C, fillna=True)
df['ADX'] = ta.trend.adx(high=H, low=L, close=C, fillna=True)
df['-VI'] = ta.trend.vortex_indicator_neg(
    high=H, low=L, close=C, fillna=True)
df['+VI'] = ta.trend.vortex_indicator_pos(
    high=H, low=L, close=C, fillna=True)
df['TRIX'] = ta.trend.trix(close=C, fillna=True)
df['MI'] = ta.trend.mass_index(high=H, low=L, fillna=True)
df['CCI'] = ta.trend.cci(high=H, low=L, close=C, fillna=True)
df['DPO'] = ta.trend.dpo(close=C, fillna=True)
df['KST'] = ta.trend.kst(close=C, fillna=True)
df['Ichimoku'] = ta.trend.ichimoku_a(high=H, low=L, fillna=True)
df['Parabolic SAR'] = ta.trend.psar_down(
    high=H, low=L, close=C, fillna=True)
df['STC'] = ta.trend.stc(close=C, fillna=True)
# Momentum
df['RSI'] = ta.momentum.rsi(close=C, fillna=True)
df['SRSI'] = ta.momentum.stochrsi(close=C, fillna=True)
df['TSI'] = ta.momentum.tsi(close=C, fillna=True)
df['UO'] = ta.momentum.ultimate_oscillator(
    high=H, low=L, close=C, fillna=True)
df['SR'] = ta.momentum.stoch(close=C, high=H, low=L, fillna=True)
df['WR'] = ta.momentum.williams_r(high=H, low=L, close=C, fillna=True)
df['AO'] = ta.momentum.awesome_oscillator(high=H, low=L, fillna=True)
df['KAMA'] = ta.momentum.kama(close=C, fillna=True)
df['ROC'] = ta.momentum.roc(close=C, fillna=True)
df['PPO'] = ta.momentum.ppo(close=C, fillna=True)
df['PVO'] = ta.momentum.pvo(volume=V, fillna=True)


from sklearn.preprocessing import MinMaxScaler

# 피처값 스케일링
scaler = MinMaxScaler()

scaled_df = scaler.fit_transform(df.drop(['Close'], axis=1))
scaled_df = pd.DataFrame(scaled_df, columns = df.drop(['Close'], axis=1).columns)

scaler1 = MinMaxScaler()

temp = scaler1.fit_transform(df['Close'].values.reshape(-1,1))

scaled_df.insert(3, 'Close',temp)
scaled_df

df = scaled_df



import numpy as np

def make_dataset(feature, label, window_size = 20):
  
  feature_list = []
  label_list = []
  
  for i in range(len(feature) - window_size) :
    feature_list.append(feature.iloc[i:i+window_size])
    label_list.append(label.iloc[i+window_size])
  
  return np.array(feature_list), np.array(label_list)

WINDOW_SIZE=20

result = []
predict = []


####최근 20개의 데이터만
pred_feature = df[-WINDOW_SIZE :]
pred_feature = pred_feature.to_numpy()

pred_feature = pred_feature.reshape((1,20,-1))
# pred_feature.shape

import random as rn
import tensorflow as tf

# seed_num = 42
# np.random.seed(seed_num)
# rn.seed(seed_num)
# tf.random.set_seed(seed_num)
# tf.set_random_seed(seed_num)

for i in range(1, len(df)//1000+2):
  
        # seed_num = 42
        # np.random.seed(seed_num)
        # rn.seed(seed_num)
        # tf.random.set_seed(seed_num)
        
        # 데이터 불러오기
        df_copy = df
        # df = df.set_index(keys=['Date'], inplace=False, drop=True)
        # df = df.drop('Close', axis = 1)
        # df.rename(columns ={'Adj Close':'Close'}, inplace = True)
        # 원하는 크기로 데이터 자르기
        df_copy = df_copy[-i*1000:]
        # 피처값, 타켓 스케일링
        
        TEST_SIZE = int(len(df_copy) * 0.7)
        
        train = df_copy[:TEST_SIZE]
        test = df_copy[TEST_SIZE:]

##200일의 데이터를 얼마나 잘 예측하느냐
        
        # feature_cols = df.drop('Close', axis =1).columns
        feature_cols = df.columns
        label_cols = ['Close']

        train_feature = train[feature_cols]
        train_label = train[label_cols]

        train_feature, train_label = make_dataset(train_feature, train_label,20)

        from sklearn.model_selection import train_test_split
        x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2, shuffle = False)
        
        
        test_feature = test[feature_cols]
        test_label = test[label_cols]
        test_feature , test_label = make_dataset(test_feature, test_label,20)
        
        
        from keras.layers.normalization import batch_normalization

        model = Sequential()

        model.add(GRU(16, 
                      input_shape=(train_feature.shape[1], train_feature.shape[2]), 
                      return_sequences=False,
                      activation='relu'))
      
        model.add(Dense(1))
        # model.add(Dropout(0.2, seed=123))
      
        
        model.compile(loss='mean_squared_error', optimizer='adam')

        early_stop = EarlyStopping(monitor='val_loss', patience=10)

        import os
        from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

        model_path = 'model'
        filename = os.path.join(model_path, 'tmp_checkpoint.h5')
        checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

        history = model.fit(x_train, y_train, 
                                            epochs=100, 
                                            batch_size=16,
                                            validation_data=(x_valid, y_valid), 
                                            callbacks=[early_stop, checkpoint])


        pred = model.predict(test_feature)
        # rescaleing 작업
        rescaled_y_true = scaler1.inverse_transform(np.array(test_label).reshape(-1, 1))
        rescaled_pred = scaler1.inverse_transform(np.array(pred).reshape(-1,1))
        
        
        # 평가지표(RMSE) 계산
        RMSE = np.sqrt(mean_squared_error(rescaled_y_true, rescaled_pred))
        result.append(RMSE)
        print(f"{i * 1000}길이의 데이터 적용 완료\n 길이: {i * 1000}, RMSE:{RMSE}")
        print(result)
        
        
        model.load_weights(filename)

        new_pred = model.predict(pred_feature)
        rescaled_newpred = scaler1.inverse_transform(np.array(new_pred).reshape(-1,1))
        predict.append(rescaled_newpred)
        
        # new_pred = model.predict(pred_feature)
        # rescaled_newpred = scaler1.inverse_transform(np.array(new_pred).reshape(-1,1))
        # predict.append(rescaled_newpred)
        
        
        
        
    # data_size = list(range(1000, len(df_origin), 1000))
    # data_size.append(len(df_origin))
    # plt.style.use("ggplot")
    # plt.rcParams['font.family'] = 'Malgun Gothic'
    # data_size = list(range(1000, len(df_origin), 1000))
    # data_size.append(len(df_origin))
    # plt.figure(figsize=(9, 6))
    # plt.plot(data_size, result)
    # plt.tick_params(axis='x')
    # plt.tick_params(axis='y')
    # plt.title(f'{file_name}')
    # plt.show()

C:\Users\gram\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\gram\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


Epoch 1/100
31/34 [==========================>...] - ETA: 0s - loss: 0.5314
Epoch 1: val_loss improved from inf to 0.32489, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 2s 15ms/step - loss: 0.4980 - val_loss: 0.3249
Epoch 2/100
31/34 [==========================>...] - ETA: 0s - loss: 0.0710
Epoch 2: val_loss improved from 0.32489 to 0.01770, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 0s 8ms/step - loss: 0.0655 - val_loss: 0.0177
Epoch 3/100
27/34 [======================>.......] - ETA: 0s - loss: 0.0024
Epoch 3: val_loss improved from 0.01770 to 0.00934, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 0s 8ms/step - loss: 0.0021 - val_loss: 0.0093
Epoch 4/100
27/34 [======================>.......] - ETA: 0s - loss: 7.8160e-04
Epoch 4: val_loss did not improve from 0.00934
34/34 [==============================] - 0s 8ms/step - loss: 7.5719e-04 - val_loss: 0.0112
Epoch 5/100
29/34 [=

30/34 [=========================>....] - ETA: 0s - loss: 2.0347e-04
Epoch 34: val_loss did not improve from 0.00146
34/34 [==============================] - 0s 7ms/step - loss: 2.0528e-04 - val_loss: 0.0019
Epoch 35/100
31/34 [==========================>...] - ETA: 0s - loss: 2.0401e-04
Epoch 35: val_loss did not improve from 0.00146
34/34 [==============================] - 0s 7ms/step - loss: 2.0194e-04 - val_loss: 0.0016
Epoch 36/100
31/34 [==========================>...] - ETA: 0s - loss: 2.0662e-04
Epoch 36: val_loss did not improve from 0.00146
34/34 [==============================] - 0s 7ms/step - loss: 2.0025e-04 - val_loss: 0.0018
Epoch 37/100
31/34 [==========================>...] - ETA: 0s - loss: 1.8631e-04
Epoch 37: val_loss improved from 0.00146 to 0.00139, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 0s 8ms/step - loss: 1.9260e-04 - val_loss: 0.0014
Epoch 38/100
29/34 [========================>.....] - ETA: 0s - loss: 1.9540e-04
Epoch 3

Epoch 69/100
31/34 [==========================>...] - ETA: 0s - loss: 1.4489e-04
Epoch 69: val_loss did not improve from 0.00082
34/34 [==============================] - 0s 7ms/step - loss: 1.4745e-04 - val_loss: 9.5427e-04
Epoch 70/100
30/34 [=========================>....] - ETA: 0s - loss: 1.5095e-04
Epoch 70: val_loss improved from 0.00082 to 0.00073, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 0s 8ms/step - loss: 1.4666e-04 - val_loss: 7.3230e-04
Epoch 71/100
29/34 [========================>.....] - ETA: 0s - loss: 1.5065e-04
Epoch 71: val_loss did not improve from 0.00073
34/34 [==============================] - 0s 7ms/step - loss: 1.4957e-04 - val_loss: 9.1137e-04
Epoch 72/100
31/34 [==========================>...] - ETA: 0s - loss: 1.4625e-04
Epoch 72: val_loss did not improve from 0.00073
34/34 [==============================] - 0s 7ms/step - loss: 1.4464e-04 - val_loss: 8.1880e-04
Epoch 73/100
31/34 [==========================>...] - ETA: 

Epoch 23/100
63/69 [==========================>...] - ETA: 0s - loss: 1.5947e-04
Epoch 23: val_loss did not improve from 0.00012
69/69 [==============================] - 1s 7ms/step - loss: 1.5147e-04 - val_loss: 1.3161e-04
Epoch 24/100
60/69 [=========================>....] - ETA: 0s - loss: 1.1967e-04
Epoch 24: val_loss did not improve from 0.00012
69/69 [==============================] - 0s 6ms/step - loss: 1.2096e-04 - val_loss: 1.2463e-04
Epoch 25/100
62/69 [=========================>....] - ETA: 0s - loss: 1.1836e-04
Epoch 25: val_loss did not improve from 0.00012
69/69 [==============================] - 0s 6ms/step - loss: 1.2076e-04 - val_loss: 3.2909e-04
Epoch 26/100
60/69 [=========================>....] - ETA: 0s - loss: 1.5740e-04
Epoch 26: val_loss improved from 0.00012 to 0.00012, saving model to model\tmp_checkpoint.h5
69/69 [==============================] - 0s 7ms/step - loss: 1.5201e-04 - val_loss: 1.2030e-04
Epoch 27/100
62/69 [=========================>....] - ETA: 

...

273/279 [============================>.] - ETA: 0s - loss: 1.3198e-05
Epoch 19: val_loss did not improve from 0.00003
279/279 [==============================] - 2s 6ms/step - loss: 1.3088e-05 - val_loss: 5.7877e-05
Epoch 20/100
276/279 [============================>.] - ETA: 0s - loss: 1.1780e-05
Epoch 20: val_loss did not improve from 0.00003
279/279 [==============================] - 2s 6ms/step - loss: 1.1738e-05 - val_loss: 8.8015e-05
Epoch 21/100
274/279 [============================>.] - ETA: 0s - loss: 1.2678e-05
Epoch 21: val_loss did not improve from 0.00003
279/279 [==============================] - 2s 6ms/step - loss: 1.2621e-05 - val_loss: 3.8581e-05
Epoch 22/100
278/279 [============================>.] - ETA: 0s - loss: 1.3673e-05
Epoch 22: val_loss did not improve from 0.00003
279/279 [==============================] - 2s 6ms/step - loss: 1.3645e-05 - val_loss: 3.6983e-05
Epoch 23/100
271/279 [============================>.] - ETA: 0s - loss: 1.3447e-05
Epoch 23: val_loss

314/314 [==============================] - 2s 6ms/step - loss: 1.2716e-04 - val_loss: 9.3109e-05
Epoch 3/100
310/314 [============================>.] - ETA: 0s - loss: 5.2440e-05
Epoch 3: val_loss improved from 0.00009 to 0.00006, saving model to model\tmp_checkpoint.h5
314/314 [==============================] - 2s 7ms/step - loss: 5.2267e-05 - val_loss: 6.2337e-05
Epoch 4/100
309/314 [============================>.] - ETA: 0s - loss: 3.0876e-05
Epoch 4: val_loss improved from 0.00006 to 0.00005, saving model to model\tmp_checkpoint.h5
314/314 [==============================] - 2s 6ms/step - loss: 3.0835e-05 - val_loss: 4.6733e-05
Epoch 5/100
310/314 [============================>.] - ETA: 0s - loss: 2.3688e-05
Epoch 5: val_loss improved from 0.00005 to 0.00004, saving model to model\tmp_checkpoint.h5
314/314 [==============================] - 2s 6ms/step - loss: 2.3778e-05 - val_loss: 4.4155e-05
Epoch 6/100
311/314 [============================>.] - ETA: 0s - loss: 1.8348e-05
Epoch 6:

Epoch 36/100
314/314 [==============================] - ETA: 0s - loss: 5.9261e-06
Epoch 36: val_loss improved from 0.00001 to 0.00001, saving model to model\tmp_checkpoint.h5
314/314 [==============================] - 2s 7ms/step - loss: 5.9261e-06 - val_loss: 1.3101e-05
Epoch 37/100
312/314 [============================>.] - ETA: 0s - loss: 4.9390e-06
Epoch 37: val_loss did not improve from 0.00001
314/314 [==============================] - 3s 8ms/step - loss: 4.9234e-06 - val_loss: 2.0182e-05
Epoch 38/100
308/314 [============================>.] - ETA: 0s - loss: 4.3501e-06
Epoch 38: val_loss did not improve from 0.00001
314/314 [==============================] - 2s 7ms/step - loss: 4.3757e-06 - val_loss: 1.9180e-05
Epoch 39/100
314/314 [==============================] - ETA: 0s - loss: 4.9229e-06
Epoch 39: val_loss did not improve from 0.00001
314/314 [==============================] - 2s 7ms/step - loss: 4.9229e-06 - val_loss: 1.4945e-05
Epoch 40/100
312/314 [=====================

314/314 [==============================] - 2s 7ms/step - loss: 3.0224e-06 - val_loss: 1.3658e-05
Epoch 71/100
313/314 [============================>.] - ETA: 0s - loss: 3.4295e-06
Epoch 71: val_loss did not improve from 0.00001
314/314 [==============================] - 2s 7ms/step - loss: 3.4294e-06 - val_loss: 2.1346e-05
Epoch 72/100
314/314 [==============================] - ETA: 0s - loss: 3.2242e-06
Epoch 72: val_loss did not improve from 0.00001
314/314 [==============================] - 2s 6ms/step - loss: 3.2242e-06 - val_loss: 1.1848e-05
Epoch 73/100
308/314 [============================>.] - ETA: 0s - loss: 3.5548e-06
Epoch 73: val_loss did not improve from 0.00001
314/314 [==============================] - 2s 6ms/step - loss: 3.5771e-06 - val_loss: 1.1999e-05
Epoch 74/100
312/314 [============================>.] - ETA: 0s - loss: 3.4036e-06
Epoch 74: val_loss did not improve from 0.00001
314/314 [==============================] - 2s 6ms/step - loss: 3.4040e-06 - val_loss: 1.

341/341 [==============================] - 3s 10ms/step - loss: 5.5709e-06 - val_loss: 6.4164e-05
Epoch 28/100
336/341 [============================>.] - ETA: 0s - loss: 5.6753e-06
Epoch 28: val_loss did not improve from 0.00003
341/341 [==============================] - 3s 10ms/step - loss: 5.6619e-06 - val_loss: 4.8027e-05
Epoch 29/100
340/341 [============================>.] - ETA: 0s - loss: 5.9299e-06
Epoch 29: val_loss did not improve from 0.00003
341/341 [==============================] - 4s 11ms/step - loss: 5.9231e-06 - val_loss: 5.1411e-05
Epoch 30/100
341/341 [==============================] - ETA: 0s - loss: 5.8007e-06
Epoch 30: val_loss did not improve from 0.00003
341/341 [==============================] - 4s 10ms/step - loss: 5.8007e-06 - val_loss: 5.4026e-05
Epoch 31/100
338/341 [============================>.] - ETA: 0s - loss: 5.3849e-06
Epoch 31: val_loss did not improve from 0.00003
341/341 [==============================] - 4s 12ms/step - loss: 5.3880e-06 - val_los

In [13]:
print(predict)

[array([[65861.75]], dtype=float32), array([[67188.03]], dtype=float32), array([[65368.117]], dtype=float32), array([[60948.01]], dtype=float32), array([[70028.55]], dtype=float32), array([[51181.188]], dtype=float32), array([[39066.168]], dtype=float32), array([[66275.6]], dtype=float32), array([[57670.184]], dtype=float32), array([[35160.973]], dtype=float32)]
